# LlamaIndex Async

This notebook demonstrates how to monitor Llama-index async apps with TruLens.

### Import from LlamaIndex and TruLens

In [ ]:
!pip install trulens trulens-apps-llamaindex trulens-providers-openai llama-index llama-index-readers-web openai

In [ ]:
from llama_index.core import VectorStoreIndex
from llama_index.readers.web import SimpleWebPageReader
from trulens.core import Feedback
from trulens.core import TruSession
from trulens.apps.llamaindex import TruLlama
from trulens.providers.openai import OpenAI

session = TruSession()

### Add API keys
For this example you need an OpenAI key

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "..."

### Create Async App

In [ ]:
documents = SimpleWebPageReader(html_to_text=True).load_data(
    ["http://paulgraham.com/worked.html"]
)
index = VectorStoreIndex.from_documents(documents)

query_engine = index.as_query_engine()

In [ ]:
response = query_engine.aquery("What did the author do growing up?")

print(response)  # should be awaitable
print(await response)

### Set up Evaluation

In [ ]:
# Initialize OpenAI-based feedback function collection class:
openai = OpenAI()

# Question/answer relevance between overall question and answer.
f_qa_relevance = Feedback(
    openai.relevance, name="QA Relevance"
).on_input_output()

### Create tracked app

In [ ]:
tru_query_engine_recorder = TruLlama(query_engine, feedbacks=[f_qa_relevance])

### Run Async Application with TruLens

In [ ]:
async with tru_query_engine_recorder as recording:
    response = await query_engine.aquery("What did the author do growing up?")

print(response)

record = recording.get()

In [ ]:
# Check recorded input and output:

print(record.main_input)
print(record.main_output)

In [ ]:
# Check costs:

record.cost

In [ ]:
# Check feedback results:

record.feedback_results[0].result()

In [ ]:
from trulens.dashboard import run_dashboard

run_dashboard(session)